In [1]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    #options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [3]:
driver = setWebdriver()

In [4]:
# 네이버 블로그 검색 페이지로 이동
url = "https://search.naver.com/search.naver?ssc=tab.blog.all&query=%EB%85%B8%EC%9D%B8%20%EB%B6%80%EC%96%91&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20240101to20240401"
driver.get(url)

In [5]:
# 스코롤 다운 함수 정의
def doScrollDown(whileSeconds): 
    start = datetime.datetime.now() # 스크롤 다운 시작 시간 설정
    end = start + datetime.timedelta(seconds=whileSeconds) # 스크롤 다운 종료 시간 설정
    while True:
        # 페이지 맨 아래로 스크롤 다운
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1) # 1초 대기
        
        # 종료 시간에 도달하면 반복 종료
        if datetime.datetime.now() > end:  
            break

In [6]:
doScrollDown(2)

In [7]:
# 제목과 URL을 저장할 리스트 초기화
title_list = []     # 제목을 저장할 리스트 초기화
url_list = []       # URL을 저장할 리스트 초기화

titles = driver.find_elements(By. CLASS_NAME, "title_link") # 현재  페이지에서 class가 title_link인 페이지 저장
for i, title_element in enumerate(titles):
    try:
        title_list.append(title_element.text) # titles 리스트에서 제목을 추출해 title_list에 추가
        url_list.append(title_element.get_attribute("href")) # titles 리스트에서 URL을 추출해 url_list에 추가
    except:
        print("오류 발생")
        continue
    if (i+1) % 10 == 0:
        print(f"{i+1}개 완료")  # 10개마다 완료 메시지 출력

10개 완료
20개 완료
30개 완료
40개 완료
50개 완료
60개 완료
70개 완료
80개 완료
90개 완료


In [11]:
print(f'title list({len(title_list)}): {title_list}')
print(f'url list({len(url_list)}): {url_list}')

title list(90): ['[다산 마음 요양보호사 교육원] 가족관계 변화와 노인 부양 - 요양보호사 자격증 시험 연습 문제 (빈칸 채우기)', '그러나 맞벌이 부부 증가, 핵가족화, 산업화, 수명연장 등 다양한 변화로 인해 노인부양은 더이상 가정 안의...', '[심층진단-저출산 해법없나 ②] 미래세대 노인부양에 국가 존립 위협 … 총선 후 명운 걸고 연금 개혁해야', '가족교육론 - 가족문제 조사 및 예방 프로그램 구성(치매노인 부양가족)', '애써 벌어서 노인 부양한다니....부담되나요? 새로운 기회를 잡을 방법 알려드립니다.', '2060년, 1명이 노인 1.2명을 부양해야 하는 고부담사회가 온다.', '{살롱 드 서리풀} \'유치원이 노치원으로\'..."생산인구 1명이 노인 1명 부양"', '[2024 요양보호사시험]1장. 요양보호 대상자 이해/3절. 가족관계 변화와 노인 부양', '공주시, 75세 이상 노인 부양 3대 가구에 20만원 설 효행장려금', '노인의 가난, 멋진 청춘이 부양 가족 그 당사자가 되어 가는 일 [OECD 한눈에 보는 연금 2023 보고서]', '치매가족의 부양부담과 부양부담 경감을 위한 지원방안에 대해 토론하시오 (노인복지학) DownLoad', '치매노인가족의 부양부담과 스트레스 대처방안에 대하여 설명하시오.', '그러나 맞벌이 부부 증가, 핵가족화, 산업화, 수명연장 등 다양한 변화로 인해 노인부양은 더이상 가정 안의...', '노인복지주택 부양능력 심사와 입소 여부 재결정', '고령화 사회에서 노인부양문제와 해결방안에 대해 서술하시오', '뒷모습 (노인부양)', '가정방문요양 재가센터 요양보호사 어르신 의사소견서 노인복지 / 부모님과 자녀를 다 부양하는 5060세대 ]', '[치매]경도인지장애 대처 방법과 치매노인 부양방법', '부양과 간병비 마련의 어려움', '치매 노인 가족의부양부담 색칠공부 안심타투 돌봄서비스', '<이런 법이 어딨어> 백세시대, 부양 전쟁', '부모 부양 은 누구의 몫일까?', '청약. 노부모 

In [12]:
# 본문, 좋아요 수, 댓글 수, 댓글, 이미지 수, 영상 수를 저장할 리스트 초기화
new_doc = [] # 본문을 저장할 리스트 초기화
like_cnt = [] # 좋아요 수를 저장할 리스트 초기화
comment_cnt = [] # 댓글 수를 저장할 리스트 초기화
comment_list = [] # 댓글을 저장할 리스트 초기화
img_cnt = [] # 이미지 수를 저장할 리스트 초기화
div_cnt = [] # 영상 수를 저장할 리스트 초기화

# 블로그 글 크롤링
for i in range(len(url_list)):
    url_path = url_list[i] # url 불러오기
    driver.switch_to.window(driver.window_handles[0]) # 첫 번째 탭으로 이동
    driver.execute_script("window.open('{}')".format(url_path)) # url 실행
    driver.switch_to.window(driver.window_handles[1]) # 두 번째 탭으로 이동
    
    time.sleep(1) # 1초 대기
    try:
        iframes = driver.find_elements(By.TAG_NAME, 'iframe') 
        d = '' # 댓글을 저장할 변수 초기화

          # 댓글영역의 HTML코드 가져오기
        if len(iframes) > 0:   # iframe의 존재 확인
            driver.switch_to.frame(0) # 첫번째 iframe으로 전환하여 해당 iframe의 내용을 가져옴
            html = driver.page_source #  현재 페이지의 HTML 코드를 가져와서 html 변수에 저장
            soup = BS(html, 'html.parser') # html변수에 저장된 HTML코드를 파싱하여 soup을 생성 (뎃글영역에 대한 HTML코드 포함) 

            try:
                # 블로그 본문 추출 (se-main-container 클래스를 찾아서 본문 텍스트를 가져옴)
                a = soup.find('div', class_= 'se-main-container').get_text() 

            except:
                # 블로그 본문을 찾지 못한 경우 (일반 블로그의 경우 id가 postListBody 인 요소를 본문으로 가져옴)
                a = soup.find('div', id='postListBody') 
                a = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "", str(a))  # 정규표현식을 이용하여 한글만 남김

            try:
                # 좋아요 수 추출 (u_cnt _count 클래스를 찾아서 좋아요 수 텍스트를 가져옴)
                b = soup.find('em', class_='u_cnt _count').get_text()
            except:
                # 좋아요 수를 찾지 못한 경우
                b = 'null'

            try:
                # 댓글 수 추출 (commentCount id를 찾아서 댓글 수 텍스트를 가져옴)
                c = soup.find('em', id="commentCount").get_text()
            except:
                # 댓글 수를 찾지 못한 경우
                c = 'null'

            try:
                # 댓글을 모두 보기 위해 댓글 버튼 클릭
                comment = driver.find_elements(By.CLASS_NAME, 'btn_arr')
                comment[len(comment) - 1].click()  # 마지막 댓글버튼 클릭
                time.sleep(1)

                commentLen = len(driver.find_elements(By.CLASS_NAME, 'u_cbox_page'))
                d = ''
                # 각 댓글 페이지의 댓글 추출
                for _ in range(0, commentLen) :
                    commentList = driver.find_elements(By.CLASS_NAME, 'u_cbox_text_wrap')
                    for j in range(len(commentList)):
                        commentList[j] = commentList[j].text
                soup = BS(html, 'html.parser')
                
                # 추출한 댓글들을 줄바꿈 문자로 구분하여 하나의 문자열로 저장
                d = "\n".join(commentList)

            except:
                # 댓글을 찾지 못한 경우
                d = 'null'

            # 이미지 수 추출 (se-image-resource egjs-visible 클래스를 찾아서 이미지 수를 가져옴)
            e = len(soup.find_all('img', class_='se-image-resource egjs-visible'))
            
            # 영상 수 추출 (pzp-ui-dimmed pzp-dimmed pzp-pc__dimmed 클래스를 찾아서 영상 수를 가져옴)
            f = len(soup.find_all('div', class_='pzp-ui-dimmed pzp-dimmed pzp-pc__dimmed'))

            # 추출한 데이터를 각각의 리스트에 추가
            new_doc.append(a)
            like_cnt.append(b)
            comment_cnt.append(c)
            comment_list.append(d)
            img_cnt.append(e)
            div_cnt.append(f)
            driver.switch_to.default_content() # iframe에서 기본 컨텐츠로 전환

        else:
            new_doc.append(' ') # 본문이 없을 경우 빈 문자열 추가
            like_cnt.append(' ') # 좋아요 수가 없을 경우 빈 문자열 추가
            comment_cnt.append(' ') # 댓글 수가 없을 경우 빈 문자열 추가
            comment_list.append(' ') # 댓글이 없을 경우 빈 문자열 추가
            img_cnt.append(' ') # 이미지 수가 없을 경우 빈 문자열 추가
            div_cnt.append(' ') # 영상 수가 없을 경우 빈 문자열 추가
            
    except:
        new_doc.append(' ') # 예외 발생 시 빈 문자열 추가
        like_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
        comment_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
        comment_list.append(' ') # 예외 발생 시 빈 문자열 추가
        img_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
        div_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
    
    driver.close()
    time.sleep(0.3) # 0.3초 대기
    
    # 매 10번째마다 진행 상황 출력
    if (i+1) % 10 == 0:
        print(f"진행 상황: {i+1}/{len(url_list)}")

진행 상황: 10/90
진행 상황: 20/90
진행 상황: 30/90
진행 상황: 40/90
진행 상황: 50/90
진행 상황: 60/90
진행 상황: 70/90
진행 상황: 80/90
진행 상황: 90/90


In [13]:
# 크롤링한 데이터를 데이터프레임으로 변환
raw_data = pd.DataFrame()
raw_data['title'] = title_list
raw_data['doc'] = new_doc
raw_data['like'] = like_cnt
raw_data['comment_cnt'] = comment_cnt
raw_data['comment_list'] = comment_list
raw_data['img'] = img_cnt
raw_data['div'] = div_cnt
raw_data['ch'] = 'naver'
raw_data['ch2'] = 'blog'

In [18]:
# 데이터프레임을 pickle파일로 저장
file_path = "../data/"
f = open(f'{file_path}naver_blog202403.pkl', "wb")
pickle.dump(raw_data,f)

f.close()

In [ ]:
driver.quit()

In [22]:
# 저장된 pickle 파일을 불러옴
f = open(f'{file_path}naver_blog202403.pkl', "rb")
temp_file = pickle.load(f)
f.close()

temp_file

,title,doc,like,comment_cnt,comment_list,img,div,ch,ch2
0,[다산 마음 요양보호사 교육원] 가족관계 변화와 노인 부양 - 요양보호사 자격증 시...,,,,,,,naver,blog
1,"그러나 맞벌이 부부 증가, 핵가족화, 산업화, 수명연장 등 다양한 변화로 인해 노인...",,,,,,,naver,blog
2,[심층진단-저출산 해법없나 ②] 미래세대 노인부양에 국가 존립 위협 … 총선 후 명...,,,,,,,naver,blog
3,가족교육론 - 가족문제 조사 및 예방 프로그램 구성(치매노인 부양가족),,,,,,,naver,blog
4,애써 벌어서 노인 부양한다니....부담되나요? 새로운 기회를 잡을 방법 알려드립니다.,,,,,,,naver,blog
...,...,...,...,...,...,...,...,...,...
85,"[소풍, 플랜75]: 죽음을 강제당하는 노인들",,,,,,,naver,blog
86,"보건복지부, 노인 천만 시대를 맞이한 2024년 노인보건복지 사업안내 제작 배부",,,,,,,naver,blog
87,"강서구 방문요양/ 만성질환을 가진 노인의 배우자는... 방문요양 강서화곡센터, 노인...",,,,,,,naver,blog
88,청년버팀목전세자금대출 후기 나이 조건,,,,,,,naver,blog


In [23]:
# CSV파일로 저장
temp_file.to_csv("naver_blog.csv", index = False, encoding="utf-8-sig")

## Naver Cafe Crawling

In [38]:
driver = setWebdriver()

query = input("검색할 키워드를 입력하세요: ")
start_date = re.sub(r'[^0-9]', '', input("검색 시작 날짜를 입력하세요(예: 2024.02.01): "))
end_date = re.sub(r'[^0-9]', '',input("검색 종료 날짜를 입력하세요(예: 2024.04.01): "))

url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query={query}&ie=utf8&st=rel&date_option=8&date_from={start_date}&date_to={end_date}&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Add%2Cp%3Afrom20240201to20240401&nso_open=1&prdtype=0'
driver.get(url)

In [39]:
title_list = [] # 제목을 저장할 리스트 초기화
url_list = [] # URL을 저장할 리스트 초기화

titles = driver.find_elements(By.CLASS_NAME, "title_link") # 현재 페이지에서 class가 title_link인 페이지 저장
for i, title_element in enumerate(titles):
    try:
        title_list.append(title_element.text) # titles 리스트에서 제목을 추출해 title_list에 추가
        url_list.append(title_element.get_attribute("href")) # titles 리스트에서 URL을 추출해 url_list에 추가
    except:
        print("오류 발생")
        continue

    if (i+1) % 10 == 0:
        print(f"{i+1}개 완료")

10개 완료
20개 완료
30개 완료


In [42]:
new_doc = []
comment_cnt = []
comment_list = []
img_cnt = []
div_cnt = []

# 블로그 글 크롤링
for i in range(len(url_list)):
    url_path = url_list[i] # url 불러오기
    driver.switch_to.window(driver.window_handles[0]) # 첫 번째 탭으로 이동
    driver.execute_script("window.open('{}')".format(url_path)) # url 실행
    driver.switch_to.window(driver.window_handles[1]) # 두 번째 탭으로 이동

    time.sleep(rd.uniform(1, 2)) # 1~2초 대기
    try:
        frame = driver.find_elements(By.CLASS_NAME, 'disable-select')
        comments = '' # 댓글을 저장할 변수 초기화

        # 댓글영역의 HTML코드 가져오기
        if len(frame) > 0:  # frame 존재 확인
            driver.switch_to.frame(0) # 첫번째 frame으로 전환하여 해당 frame의 내용을 가져옴
            html = driver.page_source   # 현재 페이지의 HTML 코드를 가져와서 html 변수에 저장
            soup = bs(html, 'html.parser')  # html 변수에 저장된 HTML코드를 파싱하여 soup을 생성 (뎃글영역에 대한 HTML코드 포함)

            # 블로그 본문 추출
            try:
                main_text = soup.find('div', class_='se-module se-module-text').get_text()
                print(f'main_text: {main_text}')
            except: 
                main_text = None

            # 좋아요 수 추출
            try:
                like = soup.find('em', class_='u_cnt _count').get_text()
                print(f'like: {like}')
            except:
                like = None

            # 댓글 수 추출
            try:
                commentLen = soup.find('strong', id='num').get_text()
                print(f'commentLen: {commentLen}')
            except:
                commentLen = None

            # 댓글
            try:
                comment = soup.find_all('span', class_='text_comment')
                print(f'comment: {comment}')
            except:
                comment = None

            break
            
            

    except:
        driver.close()

comment: []


# 합치기